In [14]:
ENV["GKS_ENCODING"] = "utf-8"
using NLsolve
using Plots
using IntervalArithmetic, IntervalRootFinding
using SharedArrays


using HDF5
using LadderDGA
using Suppressor
using Logging

using Distributed
if nprocs() == 1
    addprocs(7)
    @everywhere using Pkg
    @everywhere Pkg.activate("/home/julian/Hamburg/LadderDGA.jl")
    @everywhere using LadderDGA
end

cfg_file = "/home/julian/Stuttgart/error_bz/config.toml"

io = stdout
metafmt(level::Logging.LogLevel, _module, group, id, file, line) = Logging.default_metafmt(level, nothing, group,     id, nothing, nothing)
logger = ConsoleLogger(io, Logging.Info, meta_formatter=metafmt, show_limited=true, right_justify=0)
global_logger(logger);

In [15]:
mP, sP, env, kGrids, qGridLoc, freqList = readConfig(cfg_file)
kG = kGrids[1]
νGrid, sumHelper_f, impQ_sp, impQ_ch, GImp_fft, GLoc_fft, Σ_loc, FUpDo, gImp, gLoc = setup_LDGA(kG, freqList, mP, sP, env);

[ Info: Reading Inputs...
[ Warning: TODO: check beta consistency, config <-> g0man, chi_dir <-> gamma dir
[ Info: setting usable ranges of sp and ch channel from 62:98 and 62:98 to the same range of 62:98
┌ Info: Inputs Read. Starting Computation.
│ Local susceptibilities with ranges are:
│ χLoc_sp(62:98) = 0.3003, χLoc_ch(62:98) = 0.0917 
└ sum χupup check (fit, tail sub, tail sub + fit, expected): 0.19596777560898987 ?≈? 0.1959677756089899 ?=? 0.1959677756089899 ?≈? 0.25"


In [16]:
# "local" correction quantities
bubbleLoc = calc_bubble(νGrid, GImp_fft, qGridLoc, mP, sP);
locQ_sp = calc_χ_trilex(impQ_sp.Γ, bubbleLoc, qGridLoc, νGrid, sumHelper_f, mP.U, mP, sP);
locQ_ch = calc_χ_trilex(impQ_ch.Γ, bubbleLoc, qGridLoc, νGrid, sumHelper_f, -mP.U, mP, sP);
Σ_ladderLoc = calc_Σ(locQ_sp, locQ_ch, bubbleLoc, GImp_fft, FUpDo,
                     qGridLoc, sumHelper_f, mP, sP)

Σ_ladderLoc = Σ_ladderLoc .+ mP.n * mP.U/2.0;
println("non local")
# ladder quantities
bubble = calc_bubble(νGrid, GLoc_fft, kG, mP, sP);
nlQ_sp = calc_χ_trilex(impQ_sp.Γ, bubble, kG, νGrid, sumHelper_f, mP.U, mP, sP);
nlQ_ch = calc_χ_trilex(impQ_ch.Γ, bubble, kG, νGrid, sumHelper_f, -mP.U, mP, sP);

nlQ_sp_nλ = deepcopy(nlQ_sp);
nlQ_ch_nλ = deepcopy(nlQ_ch);

λ_sp, λ_spch  = λ_correction!(impQ_sp, impQ_ch, FUpDo, Σ_loc, Σ_ladderLoc, nlQ_sp, nlQ_ch, bubble, GLoc_fft, kG, mP, sP)

Σ_ladder = calc_Σ(nlQ_sp, nlQ_ch, bubble, GLoc_fft, FUpDo, kG, sumHelper_f, mP, sP)
Σ_ladder_corrected = Σ_ladder .- Σ_ladderLoc .+ Σ_loc[1:size(Σ_ladder,1)];

[ Warning: Cutting off Σ_ω by hand, fix this!
non local
┌ Info: Computing λsp corrected χsp, using 
│   sP.χFillType = lambda_χ_fill::ChiFillType = 1
└    as fill value outside usable ω range. = " as fill value outside usable ω range."
[ Warning: currently using min(usable_sp, usable_ch) = min(62:98,62:98) = 62:98 for all calculations. relax this?
┌ Info: Found usable intervals for non-local susceptibility of length 
│ sp: 62:98, length: 37
│ ch: 62:98, length: 37
│ usable: 62:98, length: 37
└ χch sum = 0.08968312793102405, rhs = 0.3022524232869557
┌ Info: found 
│   χ_min = -0.963274387407372
└   ". Looking for roots in intervall $(int)" = ". Looking for roots in intervall [-0.963274387407372, 2.8700589459259613]"
┌ Info: Found λsp 
└   λsp = 0.10617635020546747
[ Info: new lambda correction: λsp=0.0 and λch=0.0
[ Warning: Cutting off Σ_ω by hand, fix this!


In [17]:
bs = sum(bubble, dims=3)[:,:,1];

In [18]:
heatmap(real(reshape(bs[1,:],(10,10))))

LoadError: DimensionMismatch("new dimensions (10, 10) must be consistent with array size 400")

In [19]:
heatmap(real(reshape(kG.ϵkGrid,(10,10))))

LoadError: DimensionMismatch("new dimensions (10, 10) must be consistent with array size 400")